In [1]:
import pandas as pd 
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
warnings.simplefilter("ignore")
sns.set_style('darkgrid')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score

In [3]:
df = pd.read_csv('dataset/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')
df.head()

,S.NO,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
0,1,Mike Tyson,USA,1,NaN,boxing,1990,28.6
1,2,Buster Douglas,USA,2,NaN,boxing,1990,26.0
2,3,Sugar Ray Leonard,USA,3,NaN,boxing,1990,13.0
3,4,Ayrton Senna,Brazil,4,NaN,auto racing,1990,10.0
4,5,Alain Prost,France,5,NaN,auto racing,1990,9.0


In [4]:
df.shape

(301, 8)

In [5]:
df.isnull().sum()

S.NO                     0
Name                     0
Nationality              0
Current Rank             0
Previous Year Rank      24
Sport                    0
Year                     0
earnings ($ million)     0
dtype: int64

In [9]:
df.drop(['Previous Year Rank'], axis=1, inplace=True)

In [6]:
# Final value = Initial value * CPI final/CPI initial
inf_df = {'cpi_per_year': [53.2, 56.45, 58.18, 59.87, 61.51, 63.16, 64.76, 66.92, 68.05, 
                          69.15, 71.01, 73.41, 74.55, 76.32, 77.76, 80.29, 83.03, 85.14, 88.62, 
                          88.7, 91.11, 92.47, 95.21, 96.87, 98.33, 99.07, 99.79, 101.86, 104.01, 106],
         'year': [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
                  2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                 2018, 2019]}
inf_df = pd.DataFrame(inf_df, columns=['year', 'cpi_per_year'])
inf_df.head()

,year,cpi_per_year
0,1990,53.20
1,1991,56.45
2,1992,58.18
3,1993,59.87
4,1994,61.51


In [7]:
npv = []
for y, d in zip(df['Year'], df['earnings ($ million)']): 
    for y_, c_ in zip(inf_df['year'], inf_df['cpi_per_year']):
        if y == y_: 
            d = d*((inf_df.at[29, 'cpi_per_year'])/c_)
            npv.append(round(d, 2))

df = df.loc[:290, :]

,S.NO,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
0,1,Mike Tyson,USA,1,NaN,boxing,1990,28.6
1,2,Buster Douglas,USA,2,NaN,boxing,1990,26.0
2,3,Sugar Ray Leonard,USA,3,NaN,boxing,1990,13.0
3,4,Ayrton Senna,Brazil,4,NaN,auto racing,1990,10.0
4,5,Alain Prost,France,5,NaN,auto racing,1990,9.0


In [8]:
df['npv'] = npv
df.head()

,S.NO,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million),npv
0,1,Mike Tyson,USA,1,NaN,boxing,1990,28.6,56.98
1,2,Buster Douglas,USA,2,NaN,boxing,1990,26.0,51.80
2,3,Sugar Ray Leonard,USA,3,NaN,boxing,1990,13.0,25.90
3,4,Ayrton Senna,Brazil,4,NaN,auto racing,1990,10.0,19.92
4,5,Alain Prost,France,5,NaN,auto racing,1990,9.0,17.93


In [10]:
# converting each word in Sport column into lower case
def lower_case(word):
    return word.lower()
df['Sport'] = df['Sport'].map(lower_case)
df.Sport.unique()

array(['boxing', 'auto racing', 'golf', 'basketball', 'tennis', 'nfl',
       'nba', 'baseball', 'ice hockey', 'american football / baseball',
       'f1 motorsports', 'nascar', 'hockey', 'auto racing (nascar)',
       'f1 racing', 'american football', 'soccer', 'cycling',
       'motorcycle gp', 'mma'], dtype=object)

In [11]:
lbl_enc = LabelEncoder()
df['Sport'] = lbl_enc.fit_transform(df['Sport'])

In [13]:
X = df[['Current Rank', 'Sport', 'Year']].values
y = df[['npv']].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [14]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, y_train)
pred = rf_reg.predict(X_test)

In [16]:
print("Mean Absolute Error: ", mean_absolute_error(y_test, pred))
print("R2 Score: ", r2_score(y_test, pred))

Mean Absolute Error:  4.176023728813555
R2 Score:  0.9480164906003279
